In [11]:
import torch
import soundfile as sf
from engine import EmoNet, EmoDS, build_emodb, benchmark
from config import cfg  # assigns the 10 emodb speakers: 8 in ds_train / 1 in ds_val / 1 in ds_test

db = build_emodb(cfg.data_dir)  # list of pairs [wav_file, emotion_id]

ds_val = EmoDS(db=db, data_dir=cfg.data_dir, split='val')  # all recording of val speaker (see cfg.speaker_assign)
ds_test = EmoDS(db=db, data_dir=cfg.data_dir, split='test')  # all recordings of the test speaker

model = EmoNet().to(cfg.dev)  # architecture inspired by wav2letter
model.load_state_dict(torch.load(cfg.best_pth))  # load best model
model.eval();

# # test
# print('\n', *(12*[' ']), 'Confusion ds_test (PREDS x TRUE)\n')
# stats = benchmark(model, ds=ds_test, dev=cfg.dev)
# print(stats['confusion'])
print(f'\nSER Accuracy ds_test (speaker {stats["speaker"]}) =', stats['accuracy'],'\n')

# val
stats = benchmark(model, ds=ds_val, dev=cfg.dev)
print(f'SER Accuracy ds_val (speaker {stats["speaker"]}) =', stats['accuracy'])

# predict emotion of my own voice
wav_file = 'fear_sadness.wav'  # 16,000 Hz
print('WAVFILE                        PREDICTED EMOTION\n')
x, _ = sf.read(filename)
x = torch.as_tensor(x[None, None, :], device=cfg.dev, dtype=torch.float)
logits = model(x)
emotion = cfg.id2name[int(logits.argmax())]
print(f'{wav_file:<20} {emotion:>20}')

ds_val length=56
ds_test length=71

SER Accuracy ds_test (speaker [15]) = 0.7678571428571429 

SER Accuracy ds_val (speaker [15]) = 0.7678571428571429
WAVFILE                        PREDICTED EMOTION

fear_sadness.wav                    anger
